In [4]:
import os

In [5]:
%pwd

'e:\\MLOPS\\Datascienceproject\\research'

In [6]:
os.chdir("../")
%pwd

'e:\\MLOPS\\Datascienceproject'

In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [44]:
from src.Datascienceproject.constants import *
from src.Datascienceproject.utils.common import read_yaml, create_directories


In [45]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)   

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

    

       

In [46]:
import os
import urllib.request as request
from src.Datascienceproject import logger
import zipfile



In [47]:
## component: data ingestion
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with File folowing info: {headers}")
        else:
            logger.info(f"File already exists of path")
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file to the data directory
        fuction returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

    

In [49]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()


except Exception as e:
    raise e

[2025-08-27 15:36:11,944 - Datasciencelogger - INFO - created directory at: artifacts]
[2025-08-27 15:36:11,947 - Datasciencelogger - INFO - created directory at: artifacts/data_ingestion]


[2025-08-27 15:36:13,221 - Datasciencelogger - INFO - artifacts/data_ingestion/data.zip download! with File folowing info: Connection: close
Content-Length: 21952
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "a9d75aab9e75da6b7d94425ffad563d4658ff5a867321db65a954c2f2322d9e3"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DE45:E0B5B:A459D:1B70A4:68AED894
Accept-Ranges: bytes
Date: Wed, 27 Aug 2025 10:06:12 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10222-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1756289172.142584,VS0,VE248
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: d33f4ae7e4a1fea38dd63e6639cb1723139479c5
Expires: Wed, 27 Aug 2025 10:11:12 GMT
Source-Age: 0

]
